In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2

In [ ]:
def occlude(top, bottom, distance=1e-6):
    try:
        return bottom.difference(top)
    except TopologicalError:
        return bottom.buffer(distance).difference(top.buffer(distance))

In [ ]:
class ParticleCluster(object):
    
    def __init__(
        self,
        pos,
        perlin_grid,
    ):
        self.pos = Point(pos)
        self.pg = perlin_grid
        self.particles = []
        
    def gen_start_pts_gaussian(
            self,
            n_particles=10,
            xloc=0.,
            xscale=1.,
            yloc=0.,
            yscale=1.,
        ):
        xs = self.pos.x + ss.norm(loc=xloc, scale=xscale).rvs(n_particles)
        ys = self.pos.y + ss.norm(loc=yloc, scale=yscale).rvs(n_particles)
        self.start_pts = [Point((x,y)) for x,y in zip(xs, ys)]
        
    def init_particles(self, start_bounds=None):
        for pt in self.start_pts:
            p = gp.Particle(pos=pt, grid=self.pg)
            if start_bounds == None:
                self.particles.append(p)
            elif start_area.contains(p.pos):
                self.particles.append(p)
                
    @functools.singledispatchmethod           
    def step(self, n_steps):
        for p,n in zip(self.particles, n_steps):
            for i in range(n):
                p.step()
    
    @step.register
    def _(self, n_steps: int):
        n_steps = [n_steps] * len(self.particles)
        for p,n in zip(self.particles, n_steps):
                for i in range(n):
                    p.step()
                    
    @property
    def lines(self):
        return MultiLineString([p.line for p in self.particles])
    

In [ ]:
@dataclass
class ScaleTransPrms(gp.DataClassBase):
    
    n_iters: int = 100
    d_buffer: float = -0.25
    d_translate_factor: float = 0.9
    d_translate: float = None
    angles: float = 0.
    d_translates: list = field(default=None, init=False)
    def __post_init__(self):
        self.d_buffers = np.array([self.d_buffer] * self.n_iters)
        
        if self.d_translates == None:
            if self.d_translate != None:
                self.d_translates =  np.array([self.d_translate] * self.n_iters)
            else:
                self.d_translates = self.d_buffers * self.d_translate_factor
    
    @property
    def prms(self):
        varnames = ['d_buffers', 'd_translates', 'angles']
        return {var: getattr(self, var) for var in varnames}

In [ ]:
def individual_difference(multipolygon0, multipolygon1, dist=1e-6):
    diffs = []
    for p0, p1 in itertools.product(multipolygon0, multipolygon1):
        if p0.overlaps(p1):
            try:
                diff = p0.difference(p1)
            except TopologicalError:
                diff = p0.buffer(dist).difference(p1.buffer(dist))
            diffs.append(diff)
    return diffs

## try 1

In [ ]:
page_x_inches: float = 11. # inches
page_y_inches: float = 8.5 # inches
border:float = 0.

perlin_grid_params = {
    'xstep':3,
    'ystep':3,
    'lod':10,
    'falloff':None,
    'noise_scale':0.0073,
    'noiseSeed':6
}

particle_init_grid_params = {
    'xstep':16,
    'ystep':16,
}

buffer_style = 2

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds
brad = np.min([gp.get_width(drawbox), gp.get_height(drawbox)])
pg = gp.PerlinGrid(drawbox, **perlin_grid_params)

In [ ]:
start_area = sa.scale(drawbox.centroid.buffer(brad*0.45), xfact=1.3)

In [ ]:
start_area = drawbox.buffer(-20)

In [ ]:
xcs, ycs = gp.overlay_grid(start_area, xstep=19, ystep=15)
start_pts = [Point(x,y) for x,y in itertools.product(xcs, ycs)]
start_pts = [p for p in start_pts if start_area.contains(p)]

In [ ]:
gpolys = []
for p in start_pts:

    pc = ParticleCluster(pos=p, perlin_grid=pg)
    pc.gen_start_pts_gaussian(n_particles=20, xscale=4, yscale=4)
    pc.init_particles()
    n_steps = np.random.randint(low=10, high=50, size=len(pc.particles))
    pc.step(n_steps)

    buffer_distances = np.random.uniform(low=0.3, high=1.3) + np.random.uniform(low=0., high=0.7, size=len(pc.lines))
    polys = []
    for line, bd in zip(pc.lines, buffer_distances):
        poly = line.buffer(bd,
            cap_style=buffer_style,
            join_style=buffer_style)
        polys.append(poly)

    P = gp.Poly(so.unary_union(polys))

    stp = ScaleTransPrms(
        d_buffer=np.random.uniform(low=-0.45, high=-0.25), 
        angles=-90,

    )
    stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)
    P.fill_scale_trans(**stp.prms)
    gpolys.append(P)

In [ ]:
zorder = np.random.permutation(len(gpolys))

for z, gpoly in zip(zorder, gpolys):
    gpoly.z = z

for gp0, gp1 in itertools.combinations(gpolys, r=2):
    try:
        overlaps = gp0.p.overlaps(gp1.p)
    except :
        gp0.p = gp0.p.buffer(1e-6)
        gp1.p = gp1.p.buffer(1e-6)
        overlaps = gp0.p.overlaps(gp1.p)
    if overlaps:
        if gp0.z > gp1.z:
            gp1.p = occlude(top=gp0.p, bottom=gp1.p)
        elif gp0.z < gp1.z:
            gp0.p = occlude(top=gp1.p, bottom=gp0.p)

In [ ]:
ifills = []
for p in gpolys:
    try:
        ifills.append(p.intersection_fill)
    except:
        pass

splits = utils.random_split(ifills, n_layers=5)
layers = [utils.merge_LineStrings(split) for split in splits]

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)
sk.display(color_mode='layer')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0083_perlin_flow_erode_frays_occlude.svg'

sk.save(savepath)

vpype_commands = 'linesimplify --tolerance 0.01mm linemerge --tolerance 0.1mm reloop linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write --page-format {page_format} {savepath}'

os.system(vpype_str)

## try 2

In [ ]:
page_x_inches: float = 11. # inches
page_y_inches: float = 8.5 # inches
border:float = 0.

perlin_grid_params = {
    'xstep':1,
    'ystep':1,
    'lod':10,
    'falloff':None,
    'noise_scale':0.0073,
    'noiseSeed':8
}
buffer_style = 2

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds
brad = np.min([gp.get_width(drawbox), gp.get_height(drawbox)])
pg = gp.PerlinGrid(drawbox, **perlin_grid_params)

In [ ]:
start_area = drawbox

In [ ]:
xcs, ycs = gp.overlay_grid(start_area, xstep=15, ystep=15)
start_pts = [Point(x,y) for x,y in itertools.product(xcs, ycs)]
start_pts = [p for p in start_pts if start_area.contains(p)]

In [ ]:
gpolys = []
for p in start_pts:

    pc = ParticleCluster(pos=p, perlin_grid=pg)
    pc.gen_start_pts_gaussian(n_particles=30, xscale=7, yscale=7)
    pc.init_particles(start_bounds=drawbox)
    n_steps = np.random.randint(low=10, high=90, size=len(pc.particles))
    pc.step(n_steps)

    buffer_distances = np.random.uniform(low=0.3, high=1.3) + np.random.uniform(low=0., high=0.7, size=len(pc.lines))
    polys = []
    for line, bd in zip(pc.lines, buffer_distances):
        poly = line.buffer(bd,
            cap_style=buffer_style,
            join_style=buffer_style)
        polys.append(poly)

    P = gp.Poly(so.unary_union(polys))

    stp = ScaleTransPrms(
        d_buffer=np.random.uniform(low=-0.4, high=-0.2),
        d_translate_factor=0.7,
        angles=-240,

    )
    stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)
    P.fill_scale_trans(**stp.prms)
    gpolys.append(P)

In [ ]:
zorder = np.random.permutation(len(gpolys))

for z, gpoly in zip(zorder, gpolys):
    gpoly.z = z

for gp0, gp1 in itertools.combinations(gpolys, r=2):
    try:
        overlaps = gp0.p.overlaps(gp1.p)
    except:
        gp0.p = gp0.p.buffer(1e-6)
        gp1.p = gp1.p.buffer(1e-6)
        overlaps = gp0.p.overlaps(gp1.p)
    if overlaps:
        if gp0.z > gp1.z:
            gp1.p = occlude(top=gp0.p, bottom=gp1.p)
        elif gp0.z < gp1.z:
            gp0.p = occlude(top=gp1.p, bottom=gp0.p)

In [ ]:
ifills = []
for p in gpolys:
    ifills.append(p.intersection_fill)

In [ ]:
splits = utils.random_split(ifills, n_layers=4)
layers = [utils.merge_LineStrings(split) for split in splits]

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)
sk.display(color_mode='layer')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0084_perlin_flow_erode_frays_occlude.svg'

sk.save(savepath)

vpype_commands = 'linesimplify --tolerance 0.05mm linemerge --tolerance 0.1mm reloop linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write --page-format {page_format} {savepath}'

os.system(vpype_str)

## try 3

In [ ]:
page_x_inches: float = 6 # inches
page_y_inches: float = 6 # inches
border:float = 20.

perlin_grid_params = {
    'xstep':1,
    'ystep':1,
    'lod':10,
    'falloff':None,
    'noise_scale':0.0063,
    'noiseSeed':8
}
buffer_style = 2

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds
brad = np.min([gp.get_width(drawbox), gp.get_height(drawbox)])
pg = gp.PerlinGrid(drawbox, **perlin_grid_params)
start_area = drawbox
xcs, ycs = gp.overlay_grid(start_area, xstep=45, ystep=45)
start_pts = [Point(x,y) for x,y in itertools.product(xcs, ycs)]
start_pts = [p for p in start_pts if start_area.contains(p)]

In [ ]:
gpolys = []
for p in start_pts:

    pc = ParticleCluster(pos=p, perlin_grid=pg)
    pc.gen_start_pts_gaussian(n_particles=30, xscale=6, yscale=6)
    pc.init_particles(start_bounds=drawbox)
    n_steps = np.random.randint(low=80, high=190, size=len(pc.particles))
    pc.step(n_steps)

    buffer_distances = np.random.uniform(low=0.3, high=1.3) + np.random.uniform(low=0., high=0.7, size=len(pc.lines))
    polys = []
    for line, bd in zip(pc.lines, buffer_distances):
        poly = line.buffer(bd,
            cap_style=buffer_style,
            join_style=buffer_style)
        polys.append(poly)

    P = gp.Poly(so.unary_union(polys))
    
    stp = ScaleTransPrms(
        d_buffer=np.random.uniform(low=-0.4, high=-0.2),
        d_translate_factor=0.7,
        angles=-240,

    )
    stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)
    P.fill0 = gp.merge_LineStrings([p.boundary for p in gp.scale_trans(P.p, **stp.prms)])
    
    stp = ScaleTransPrms(
        d_buffer=np.random.uniform(low=-0.4, high=-0.2),
        d_translate_factor=0.7,
        angles=120,

    )
    stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)
    P.fill1 = gp.merge_LineStrings([p.boundary for p in gp.scale_trans(P.p, **stp.prms)])
    gpolys.append(P)

In [ ]:
zorder = np.random.permutation(len(gpolys))

for z, gpoly in zip(zorder, gpolys):
    gpoly.z = z

for gp0, gp1 in itertools.combinations(gpolys, r=2):
    try:
        overlaps = gp0.p.overlaps(gp1.p)
    except:
        gp0.p = gp0.p.buffer(1e-6)
        gp1.p = gp1.p.buffer(1e-6)
        overlaps = gp0.p.overlaps(gp1.p)
    if overlaps:
        if gp0.z > gp1.z:
            gp1.p = occlude(top=gp0.p, bottom=gp1.p)
        elif gp0.z < gp1.z:
            gp0.p = occlude(top=gp1.p, bottom=gp0.p)

In [ ]:
ifills0 = []
ifills1 = []
for p in gpolys:
    ifills0.append(p.fill0.intersection(p.p.buffer(1e-6)))
    ifills1.append(p.fill1.intersection(p.p.buffer(1e-6)))

In [ ]:
layers = []
layers.append(gp.merge_LineStrings(ifills0))
layers.append(gp.merge_LineStrings(ifills1))

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)
sk.display(color_mode='layer')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0085_perlin_flow_erode_frays_color_mix.svg'

sk.save(savepath)

vpype_commands = 'linesimplify --tolerance 0.2mm linemerge --tolerance 0.2mm reloop linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write --page-format {page_format} {savepath}'

os.system(vpype_str)

## try 4 for fabiano black black

In [ ]:

border:float = 25.

perlin_grid_params = {
    'xstep':1,
    'ystep':1,
    'lod':10,
    'falloff':None,
    'noise_scale':0.0063,
    'noiseSeed':8
}
buffer_style = 2

In [ ]:
px = 200
py = 200
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds
brad = np.min([gp.get_width(drawbox), gp.get_height(drawbox)])

start_area = drawbox.centroid.buffer(brad/2)
pg = gp.PerlinGrid(start_area, **perlin_grid_params)
xcs, ycs = gp.overlay_grid(start_area, xstep=11, ystep=11)
start_pts = [Point(x,y) for x,y in itertools.product(xcs, ycs)]
start_pts = [p for p in start_pts if start_area.contains(p)]

In [ ]:
gpolys = []
for p in start_pts:

    pc = ParticleCluster(pos=p, perlin_grid=pg)
    pc.gen_start_pts_gaussian(n_particles=35, xscale=4, yscale=4)
    pc.init_particles(start_bounds=drawbox)
    n_steps = np.random.randint(low=10, high=60, size=len(pc.particles))
    pc.step(n_steps)

    buffer_distances = np.random.uniform(low=0.3, high=1.7) + np.random.uniform(low=0., high=0.7, size=len(pc.lines))
    polys = []
    for line, bd in zip(pc.lines, buffer_distances):
        poly = line.buffer(bd,
            cap_style=buffer_style,
            join_style=buffer_style)
        polys.append(poly)

    P = gp.Poly(so.unary_union(polys))
    
    stp = ScaleTransPrms(
        d_buffer=np.random.uniform(low=-0.45, high=-0.25),
        d_translate_factor=0.7,
        angles=-240,

    )
    stp.d_buffers += np.random.uniform(-0.06, 0.06, size=stp.d_buffers.shape)
    P.fill0 = gp.merge_LineStrings([p.boundary for p in gp.scale_trans(P.p, **stp.prms)])
    
    
    gpolys.append(P)

In [ ]:
zorder = np.random.permutation(len(gpolys))

for z, gpoly in zip(zorder, gpolys):
    gpoly.z = z

for gp0, gp1 in itertools.combinations(gpolys, r=2):
    try:
        overlaps = gp0.p.overlaps(gp1.p)
    except:
        gp0.p = gp0.p.buffer(1e-6)
        gp1.p = gp1.p.buffer(1e-6)
        overlaps = gp0.p.overlaps(gp1.p)
    if overlaps:
        if gp0.z > gp1.z:
            gp1.p = occlude(top=gp0.p, bottom=gp1.p)
        elif gp0.z < gp1.z:
            gp0.p = occlude(top=gp1.p, bottom=gp0.p)

In [ ]:
ifills0 = []
for p in gpolys:
    ifills0.append(p.fill0.intersection(p.p.buffer(1e-6)))

ifills0 = [l for l in ifills0 if l.length > 1e-1]

layers = []
layers.append(gp.merge_LineStrings(ifills0))

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)
sk.display(color_mode='layer')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0086_perlin_flow_erode_frays_color_mix.svg'

sk.save(savepath)

vpype_commands = 'linesimplify --tolerance 0.2mm linemerge --tolerance 0.2mm reloop linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write --page-format {page_format} {savepath}'

os.system(vpype_str)

## try 5 two color

In [ ]:
page_x_inches: float = 8.5 # inches
page_y_inches: float = 11 # inches
border:float = 0.

perlin_grid_params = {
    'xstep':1,
    'ystep':1,
    'lod':10,
    'falloff':None,
    'noise_scale':0.0063,
    'noiseSeed':8
}
buffer_style = 3

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds
brad = np.min([gp.get_width(drawbox), gp.get_height(drawbox)])

start_area = drawbox.centroid.buffer(brad*0.47)
pg = gp.PerlinGrid(start_area, **perlin_grid_params)
xcs, ycs = gp.overlay_grid(start_area, xstep=15, ystep=15)
start_pts = [Point(x,y) for x,y in itertools.product(xcs, ycs)]
start_pts = [p for p in start_pts if start_area.contains(p)]

In [ ]:
gpolys = []
for p in start_pts:

    pc = ParticleCluster(pos=p, perlin_grid=pg)
    pc.gen_start_pts_gaussian(n_particles=30, xscale=7, yscale=7)
    pc.init_particles(start_bounds=drawbox)
    n_steps = np.random.randint(low=10, high=100, size=len(pc.particles))
    pc.step(n_steps)

    buffer_distances = np.random.uniform(low=0.3, high=2.7) + np.random.uniform(low=0., high=0.7, size=len(pc.lines))
    polys = []
    for line, bd in zip(pc.lines, buffer_distances):
        poly = line.buffer(bd,
            cap_style=buffer_style,
            join_style=buffer_style)
        polys.append(poly)

    P = gp.Poly(so.unary_union(polys))
    
    stp = ScaleTransPrms(
        d_buffer=np.random.uniform(low=-0.5, high=-0.25),
        d_translate_factor=0.7,
        angles=np.radians(-60),

    )
    stp.d_buffers += np.random.uniform(-0.06, 0.06, size=stp.d_buffers.shape)
    P.fill0 = gp.merge_LineStrings([p.boundary for p in gp.scale_trans(P.p, **stp.prms)])
    
    stp = ScaleTransPrms(
        d_buffer=np.random.uniform(low=-0.5, high=-0.25),
        d_translate_factor=0.7,
        angles=np.radians(-120),

    )
    stp.d_buffers += np.random.uniform(-0.06, 0.06, size=stp.d_buffers.shape)
    P.fill1 = gp.merge_LineStrings([p.boundary for p in gp.scale_trans(P.p, **stp.prms)])
    
    gpolys.append(P)

In [ ]:
zorder = np.random.permutation(len(gpolys))

for z, gpoly in zip(zorder, gpolys):
    gpoly.z = z

for gp0, gp1 in itertools.combinations(gpolys, r=2):
    try:
        overlaps = gp0.p.overlaps(gp1.p)
    except:
        gp0.p = gp0.p.buffer(1e-6)
        gp1.p = gp1.p.buffer(1e-6)
        overlaps = gp0.p.overlaps(gp1.p)
    if overlaps:
        if gp0.z > gp1.z:
            gp1.p = occlude(top=gp0.p, bottom=gp1.p)
        elif gp0.z < gp1.z:
            gp0.p = occlude(top=gp1.p, bottom=gp0.p)

In [ ]:
ifills0 = []
ifills1 = []
for p in gpolys:
    ifills0.append(p.fill0.intersection(p.p.buffer(1e-6)))
    ifills1.append(p.fill1.intersection(p.p.buffer(1e-6)))
    
ifills0 = [l for l in ifills0 if l.length > 1e-1]
ifills1 = [l for l in ifills1 if l.length > 1e-1]

layers = []
layers.append(gp.merge_LineStrings(ifills0))
layers.append(gp.merge_LineStrings(ifills1))

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)
sk.display(color_mode='layer')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0087_perlin_flow_erode_frays_color_mix.svg'

sk.save(savepath)

vpype_commands = 'linesimplify --tolerance 0.2mm linemerge --tolerance 0.2mm reloop linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write --page-format {page_format} {savepath}'

os.system(vpype_str)

## try 6 three color

In [ ]:
page_x_inches: float = 8.5 # inches
page_y_inches: float = 11 # inches
border:float = 0.

perlin_grid_params = {
    'xstep':1,
    'ystep':1,
    'lod':10,
    'falloff':None,
    'noise_scale':0.0063,
    'noiseSeed':8
}
buffer_style = 3

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds
brad = np.min([gp.get_width(drawbox), gp.get_height(drawbox)])

start_area = drawbox.centroid.buffer(brad*0.47)
pg = gp.PerlinGrid(start_area, **perlin_grid_params)
xcs, ycs = gp.overlay_grid(start_area, xstep=15, ystep=15)
start_pts = [Point(x,y) for x,y in itertools.product(xcs, ycs)]
start_pts = [p for p in start_pts if start_area.contains(p)]

In [ ]:
n_fills = 3
fill_angles = [-30, -90, -150]
n_iter_choices = np.array(list(itertools.product(*[[2, 100]] * n_fills)))

In [ ]:
gpolys = []
for p in start_pts:

    pc = ParticleCluster(pos=p, perlin_grid=pg)
    pc.gen_start_pts_gaussian(n_particles=30, xscale=7, yscale=7)
    pc.init_particles(start_bounds=drawbox)
    n_steps = np.random.randint(low=10, high=100, size=len(pc.particles))
    pc.step(n_steps)

    buffer_distances = np.random.uniform(low=0.3, high=2.7) + np.random.uniform(low=0., high=0.7, size=len(pc.lines))
    polys = []
    for line, bd in zip(pc.lines, buffer_distances):
        poly = line.buffer(bd,
            cap_style=buffer_style,
            join_style=buffer_style)
        polys.append(poly)

    P = gp.Poly(so.unary_union(polys))
    P.fills = []
    n_iter_choice = n_iter_choices[np.random.choice(n_iter_choices.shape[0])]
    for i in range(n_fills):
        stp = ScaleTransPrms(
            n_iters=n_iter_choice[i],
            d_buffer=np.random.uniform(low=-0.6, high=-0.25),
            d_translate_factor=0.7,
            angles=np.radians(fill_angles[i]),)
        stp.d_buffers += np.random.uniform(-0.1, 0.1, size=stp.d_buffers.shape)
        fill = gp.merge_LineStrings([p.boundary for p in gp.scale_trans(P.p, **stp.prms)])
        P.fills.append(fill)
    
    gpolys.append(P)

In [ ]:
zorder = np.random.permutation(len(gpolys))

for z, gpoly in zip(zorder, gpolys):
    gpoly.z = z

for gp0, gp1 in itertools.combinations(gpolys, r=2):
    try:
        overlaps = gp0.p.overlaps(gp1.p)
    except:
        gp0.p = gp0.p.buffer(1e-6)
        gp1.p = gp1.p.buffer(1e-6)
        overlaps = gp0.p.overlaps(gp1.p)
    if overlaps:
        if gp0.z > gp1.z:
            gp1.p = occlude(top=gp0.p, bottom=gp1.p)
        elif gp0.z < gp1.z:
            gp0.p = occlude(top=gp1.p, bottom=gp0.p)

In [ ]:
fill_sets = []
for i in range(n_fills):
    fill_sets.append([])
    
for p in gpolys:
    for i in range(n_fills):
        fill_sets[i].append(p.fills[i].intersection(p.p.buffer(1e-6)))

layers = []
for fill_set in fill_sets:
    filter_fills = [l for l in fill_set if l.length > 0.2]
    layers.append(gp.merge_LineStrings(filter_fills))

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)
sk.display(color_mode='layer')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0088_perlin_flow_erode_frays_color_mix.svg'

sk.save(savepath)

vpype_commands = 'linesimplify --tolerance 0.2mm linemerge --tolerance 0.2mm reloop linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write --page-format {page_format} {savepath}'

os.system(vpype_str)

# try 7


In [ ]:
page_x_inches: float = 8.5 # inches
page_y_inches: float = 11 # inches
border:float = 0.

perlin_grid_params = {
    'xstep':1,
    'ystep':1,
    'lod':10,
    'falloff':None,
    'noise_scale':0.0063,
    'noiseSeed':8
}
buffer_style = 3

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds
brad = np.min([gp.get_width(drawbox), gp.get_height(drawbox)])

# start_area = drawbox.centroid.buffer(brad*0.47)
start_area = drawbox
pg = gp.PerlinGrid(start_area, **perlin_grid_params)
xcs, ycs = gp.overlay_grid(start_area, xstep=35, ystep=35)
start_pts = [Point(x,y) for x,y in itertools.product(xcs, ycs)]
start_pts = [p for p in start_pts if start_area.contains(p)]

In [ ]:
n_fills = 3
fill_angles = [-30, -90, -150]
n_iter_choices = np.array(list(itertools.product(*[[0, 100]] * n_fills)))
n_iter_choices = n_iter_choices[1:-1,:]

In [ ]:
gpolys = []
for p in start_pts:

    pc = ParticleCluster(pos=p, perlin_grid=pg)
    pc.gen_start_pts_gaussian(n_particles=60, xscale=6, yscale=6)
    pc.init_particles(start_bounds=drawbox)
    n_steps = np.random.randint(low=4, high=7, size=len(pc.particles))
    pc.step(n_steps)

    buffer_distances = np.random.uniform(low=0.3, high=0.7) + np.random.uniform(low=0., high=3.7, size=len(pc.lines))
    polys = []
    for line, bd in zip(pc.lines, buffer_distances):
        poly = line.buffer(bd,
            cap_style=buffer_style,
            join_style=buffer_style)
        polys.append(poly)

    P = gp.Poly(so.unary_union(polys))
    P.fills = []
    n_iter_choice = n_iter_choices[np.random.choice(n_iter_choices.shape[0])]
    for i in range(n_fills):
        stp = ScaleTransPrms(
            n_iters=n_iter_choice[i],
            d_buffer=np.random.uniform(low=-0.6, high=-0.25),
            d_translate_factor=0.7,
            angles=np.radians(fill_angles[i]),)
        stp.d_buffers += np.random.uniform(-0.1, 0.1, size=stp.d_buffers.shape)
        fill = gp.merge_LineStrings([p.boundary for p in gp.scale_trans(P.p, **stp.prms)])
        P.fills.append(fill)
    
    gpolys.append(P)

In [ ]:
zorder = np.random.permutation(len(gpolys))

for z, gpoly in zip(zorder, gpolys):
    gpoly.z = z

for gp0, gp1 in itertools.combinations(gpolys, r=2):
    try:
        overlaps = gp0.p.overlaps(gp1.p)
    except:
        gp0.p = gp0.p.buffer(1e-6)
        gp1.p = gp1.p.buffer(1e-6)
        overlaps = gp0.p.overlaps(gp1.p)
    if overlaps:
        if gp0.z > gp1.z:
            gp1.p = occlude(top=gp0.p, bottom=gp1.p)
        elif gp0.z < gp1.z:
            gp0.p = occlude(top=gp1.p, bottom=gp0.p)

In [ ]:
fill_sets = []
for i in range(n_fills):
    fill_sets.append([])
    
for p in gpolys:
    for i in range(n_fills):
        fill_sets[i].append(p.fills[i].intersection(p.p.buffer(1e-6)))

layers = []
for fill_set in fill_sets:
    filter_fills = [l for l in fill_set if l.length > 0.2]
    layers.append(gp.merge_LineStrings(filter_fills))

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)
sk.display(color_mode='layer')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0090_perlin_flow_erode_frays_color_mix.svg'

sk.save(savepath)

vpype_commands = 'linesimplify --tolerance 0.2mm linemerge --tolerance 0.2mm reloop linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write --page-format {page_format} {savepath}'

os.system(vpype_str)

## try 8

In [ ]:
page_x_inches: float = 8.5 # inches
page_y_inches: float = 11 # inches
border:float = 0.

perlin_grid_params = {
    'xstep':3,
    'ystep':3,
    'lod':10,
    'falloff':None,
    'noise_scale':0.0013,
    'noiseSeed':8
}
buffer_style = 3

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds
brad = np.min([gp.get_width(drawbox), gp.get_height(drawbox)])

# start_area = drawbox.centroid.buffer(brad*0.47)
start_area = drawbox.buffer(-5, cap_style=3, join_style=3)
pg = gp.PerlinGrid(start_area, **perlin_grid_params)
xcs, ycs = gp.overlay_grid(start_area, xstep=15, ystep=15)
start_pts = [Point(x,y) for x,y in itertools.product(xcs, ycs)]
start_pts = [p for p in start_pts if start_area.contains(p)]

In [ ]:
f,ax = plt.subplots(figsize=(6,6))
ax.quiver(np.cos(pg.a), np.sin(pg.a), scale=50)

In [ ]:
n_fills = 3
fill_angles = [-30, -40, -50]
n_iter_choices = np.array(list(itertools.product(*[[0, 100]] * n_fills)))
n_iter_choices = n_iter_choices[1:-1,:]

In [ ]:
gpolys = []
for p in start_pts:

    pc = ParticleCluster(pos=p, perlin_grid=pg)
    pc.gen_start_pts_gaussian(n_particles=15, xscale=3, yscale=3)
    pc.init_particles(start_bounds=drawbox)
    n_steps = np.random.randint(low=1, high=15, size=len(pc.particles))
    pc.step(n_steps)

    buffer_distances = np.random.uniform(low=0.1, high=1.7) + np.random.uniform(low=0., high=0.3, size=len(pc.lines))
    polys = []
    for line, bd in zip(pc.lines, buffer_distances):
        poly = line.buffer(bd,
            cap_style=buffer_style,
            join_style=buffer_style)
        polys.append(poly)

    P = gp.Poly(so.unary_union(polys))
    P.fills = []
    n_iter_choice = n_iter_choices[np.random.choice(n_iter_choices.shape[0])]
    for i in range(n_fills):
        stp = ScaleTransPrms(
            n_iters=n_iter_choice[i],
            d_buffer=np.random.uniform(low=-0.6, high=-0.2),
            d_translate_factor=0.8,
            angles=np.radians(fill_angles[i]),)
        stp.d_buffers += np.random.uniform(-0.1, 0.1, size=stp.d_buffers.shape)
        fill = gp.merge_LineStrings([p.boundary for p in gp.scale_trans(P.p, **stp.prms)])
        P.fills.append(fill)
    
    gpolys.append(P)

In [ ]:
zorder = np.random.permutation(len(gpolys))

for z, gpoly in zip(zorder, gpolys):
    gpoly.z = z

for gp0, gp1 in itertools.combinations(gpolys, r=2):
    try:
        overlaps = gp0.p.overlaps(gp1.p)
    except:
        gp0.p = gp0.p.buffer(1e-6)
        gp1.p = gp1.p.buffer(1e-6)
        overlaps = gp0.p.overlaps(gp1.p)
    if overlaps:
        if gp0.z > gp1.z:
            gp1.p = occlude(top=gp0.p, bottom=gp1.p)
        elif gp0.z < gp1.z:
            gp0.p = occlude(top=gp1.p, bottom=gp0.p)

In [ ]:
fill_sets = []
for i in range(n_fills):
    fill_sets.append([])
    
for p in gpolys:
    for i in range(n_fills):
        fill_sets[i].append(p.fills[i].intersection(p.p.buffer(1e-6)))

layers = []
for fill_set in fill_sets:
    filter_fills = [l for l in fill_set if l.length > 0.2]
    layers.append(gp.merge_LineStrings(filter_fills))

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)
sk.display(color_mode='layer')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0091_perlin_flow_erode_frays_color_mix.svg'

sk.save(savepath)

vpype_commands = 'reloop linesimplify --tolerance 0.2mm linemerge --tolerance 0.2mm linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write --page-format {page_format} {savepath}'

os.system(vpype_str)

# try 9


In [ ]:
page_x_inches: float = 6 # inches
page_y_inches: float = 6 # inches
border:float = 0.

perlin_grid_params = {
    'xstep':1,
    'ystep':1,
    'lod':10,
    'falloff':None,
    'noise_scale':0.0063,
    'noiseSeed':8
}
buffer_style = 3

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds
brad = np.min([gp.get_width(drawbox), gp.get_height(drawbox)])

# start_area = drawbox.centroid.buffer(brad*0.47)
start_area = drawbox.buffer(-10)
pg = gp.PerlinGrid(start_area, **perlin_grid_params)
xcs, ycs = gp.overlay_grid(start_area, xstep=25, ystep=25)
start_pts = [Point(x,y) for x,y in itertools.product(xcs, ycs)]
start_pts = [p for p in start_pts if start_area.contains(p)]

In [ ]:
n_fills = 3
fill_angles = [-30, -90, -150]
n_iter_choices = np.array(list(itertools.product(*[[0, 100]] * n_fills)))
n_iter_choices = n_iter_choices[1:-1,:]

In [ ]:
gpolys = []
for p in start_pts:

    pc = ParticleCluster(pos=p, perlin_grid=pg)
    pc.gen_start_pts_gaussian(n_particles=30, xscale=5, yscale=5)
    pc.init_particles(start_bounds=start_area)
    n_steps = np.random.randint(low=4, high=17, size=len(pc.particles))
    pc.step(n_steps)

    buffer_distances = np.random.uniform(low=0.3, high=0.7) + np.random.uniform(low=0., high=3.7, size=len(pc.lines))
    polys = []
    for line, bd in zip(pc.lines, buffer_distances):
        poly = line.buffer(bd,
            cap_style=buffer_style,
            join_style=buffer_style)
        polys.append(poly)

    P = gp.Poly(so.unary_union(polys))
    P.fills = []
    n_iter_choice = n_iter_choices[np.random.choice(n_iter_choices.shape[0])]
    for i in range(n_fills):
        stp = ScaleTransPrms(
            n_iters=n_iter_choice[i],
            d_buffer=np.random.uniform(low=-0.6, high=-0.25),
            d_translate_factor=0.7,
            angles=np.radians(fill_angles[i]),)
        stp.d_buffers += np.random.uniform(-0.1, 0.1, size=stp.d_buffers.shape)
        fill = gp.merge_LineStrings([p.boundary for p in gp.scale_trans(P.p, **stp.prms)])
        P.fills.append(fill)
    
    gpolys.append(P)

In [ ]:
zorder = np.random.permutation(len(gpolys))

for z, gpoly in zip(zorder, gpolys):
    gpoly.z = z

for gp0, gp1 in itertools.combinations(gpolys, r=2):
    try:
        overlaps = gp0.p.overlaps(gp1.p)
    except:
        gp0.p = gp0.p.buffer(1e-6)
        gp1.p = gp1.p.buffer(1e-6)
        overlaps = gp0.p.overlaps(gp1.p)
    if overlaps:
        if gp0.z > gp1.z:
            gp1.p = occlude(top=gp0.p, bottom=gp1.p)
        elif gp0.z < gp1.z:
            gp0.p = occlude(top=gp1.p, bottom=gp0.p)

In [ ]:
fill_sets = []
for i in range(n_fills):
    fill_sets.append([])
    
for p in gpolys:
    for i in range(n_fills):
        fill_sets[i].append(p.fills[i].intersection(p.p.buffer(1e-6)))

layers = []
for fill_set in fill_sets:
    filter_fills = [l for l in fill_set if l.length > 0.2]
    layers.append(gp.merge_LineStrings(filter_fills))

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)
sk.display(color_mode='layer')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0092_perlin_flow_erode_frays_color_mix.svg'

sk.save(savepath)

vpype_commands = 'reloop linesimplify --tolerance 0.2mm linemerge --tolerance 0.2mm linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write --page-format {page_format} {savepath}'

os.system(vpype_str)

# try 10

In [ ]:
page_x_inches: float = 8.5 # inches
page_y_inches: float = 11 # inches
border:float = 0.

perlin_grid_params = {
    'xstep':3,
    'ystep':3,
    'lod':10,
    'falloff':None,
    'noise_scale':0.0013,
    'noiseSeed':8
}
buffer_style = 3

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds
brad = np.min([gp.get_width(drawbox), gp.get_height(drawbox)])

# start_area = drawbox.centroid.buffer(brad*0.47)
start_area = drawbox.buffer(-5, cap_style=3, join_style=3)
pg = gp.PerlinGrid(start_area, **perlin_grid_params)
xcs, ycs = gp.overlay_grid(start_area, xstep=25, ystep=25)
start_pts = [Point(x,y) for x,y in itertools.product(xcs, ycs)]
start_pts = [p for p in start_pts if start_area.contains(p)]

In [ ]:
f,ax = plt.subplots(figsize=(6,6))
ax.quiver(np.cos(pg.a), np.sin(pg.a), scale=50)

In [ ]:
n_fills = 2
fill_angles = [-30, -40, ]
n_iter_choices = np.array(list(itertools.product(*[[0, 100]] * n_fills)))
n_iter_choices = n_iter_choices[[-1], :]

In [ ]:
gpolys = []
for p in start_pts:

    pc = ParticleCluster(pos=p, perlin_grid=pg)
    pc.gen_start_pts_gaussian(n_particles=35, xscale=5, yscale=5)
    pc.init_particles(start_bounds=drawbox)
    n_steps = np.random.randint(low=1, high=15, size=len(pc.particles))
    pc.step(n_steps)

    buffer_distances = np.random.uniform(low=0.1, high=1.7) + np.random.uniform(low=0., high=0.3, size=len(pc.lines))
    polys = []
    for line, bd in zip(pc.lines, buffer_distances):
        poly = line.buffer(bd,
            cap_style=buffer_style,
            join_style=buffer_style)
        polys.append(poly)

    P = gp.Poly(so.unary_union(polys))
    P.fills = []
    n_iter_choice = n_iter_choices[np.random.choice(n_iter_choices.shape[0])]
    for i in range(n_fills):
        stp = ScaleTransPrms(
            n_iters=n_iter_choice[i],
            d_buffer=np.random.uniform(low=-0.6, high=-0.2),
            d_translate_factor=0.8,
            angles=np.radians(fill_angles[i]),)
        stp.d_buffers += np.random.uniform(-0.1, 0.1, size=stp.d_buffers.shape)
        fill = gp.merge_LineStrings([p.boundary for p in gp.scale_trans(P.p, **stp.prms)])
        P.fills.append(fill)
    
    gpolys.append(P)

In [ ]:
zorder = np.random.permutation(len(gpolys))

for z, gpoly in zip(zorder, gpolys):
    gpoly.z = z

for gp0, gp1 in itertools.combinations(gpolys, r=2):
    try:
        overlaps = gp0.p.overlaps(gp1.p)
    except:
        gp0.p = gp0.p.buffer(1e-6)
        gp1.p = gp1.p.buffer(1e-6)
        overlaps = gp0.p.overlaps(gp1.p)
    if overlaps:
        if gp0.z > gp1.z:
            gp1.p = occlude(top=gp0.p, bottom=gp1.p)
        elif gp0.z < gp1.z:
            gp0.p = occlude(top=gp1.p, bottom=gp0.p)

In [ ]:
fill_sets = []
for i in range(n_fills):
    fill_sets.append([])
    
for p in gpolys:
    for i in range(n_fills):
        fill_sets[i].append(p.fills[i].intersection(p.p.buffer(1e-6)))

layers = []
for fill_set in fill_sets:
    filter_fills = [l for l in fill_set if l.length > 0.2]
    layers.append(gp.merge_LineStrings(filter_fills))

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)
sk.display(color_mode='layer')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0093_perlin_flow_erode_frays.svg'

sk.save(savepath)

vpype_commands = 'reloop linesimplify --tolerance 0.2mm linemerge --tolerance 0.2mm linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write --page-format {page_format} {savepath}'

os.system(vpype_str)

# try 11

In [ ]:
page_x_inches: float = 11 # inches
page_y_inches: float = 8.5 # inches
border:float = 0.

perlin_grid_params = {
    'xstep':3,
    'ystep':3,
    'lod':10,
    'falloff':None,
    'noise_scale':0.0053,
    'noiseSeed':3
}
buffer_style = 3

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds
brad = np.min([gp.get_width(drawbox), gp.get_height(drawbox)])

start_area = drawbox.buffer(-10, cap_style=3, join_style=3)
pg = gp.PerlinGrid(start_area, **perlin_grid_params)
xcs, ycs = gp.overlay_grid(start_area, xstep=25, ystep=35)
start_pts = [Point(x,y) for x,y in itertools.product(xcs, ycs)]
start_pts = [p for p in start_pts if start_area.contains(p)]

In [ ]:
f,ax = plt.subplots(figsize=(6,6))
ax.quiver(np.cos(pg.a), np.sin(pg.a), scale=50)

In [ ]:
gpolys = []
for p in start_pts:

    pc = ParticleCluster(pos=p, perlin_grid=pg)
    pc.gen_start_pts_gaussian(n_particles=40, xscale=5, yscale=5)
    pc.init_particles(start_bounds=start_area)
    n_steps = np.random.randint(low=3, high=20, size=len(pc.particles))
    pc.step(n_steps)

    buffer_distances = np.random.uniform(low=0.3, high=3.3) + np.random.uniform(low=0., high=1.7, size=len(pc.lines))
    polys = []
    for line, bd in zip(pc.lines, buffer_distances):
        poly = line.buffer(bd,
            cap_style=buffer_style,
            join_style=buffer_style)
        polys.append(poly)

    P = gp.Poly(so.unary_union(polys))

    stp = ScaleTransPrms(
        d_buffer=np.random.uniform(low=-0.75, high=-0.3), 
        angles=-90,

    )
    stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)
    P.fill_scale_trans(**stp.prms)
    gpolys.append(P)

In [ ]:
zorder = np.random.permutation(len(gpolys))

for z, gpoly in zip(zorder, gpolys):
    gpoly.z = z

for gp0, gp1 in itertools.combinations(gpolys, r=2):
    try:
        overlaps = gp0.p.overlaps(gp1.p)
    except :
        gp0.p = gp0.p.buffer(1e-6)
        gp1.p = gp1.p.buffer(1e-6)
        overlaps = gp0.p.overlaps(gp1.p)
    if overlaps:
        if gp0.z > gp1.z:
            gp1.p = occlude(top=gp0.p, bottom=gp1.p)
        elif gp0.z < gp1.z:
            gp0.p = occlude(top=gp1.p, bottom=gp0.p)

In [ ]:
ifills = []
for p in gpolys:
    try:
        ifills.append(p.intersection_fill)
    except:
        pass

splits = utils.random_split(ifills, n_layers=5)
layers = [gp.merge_LineStrings(split) for split in splits]

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)
sk.display(color_mode='layer')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0100_perlin_flow_erode_frays_occlude.svg'

sk.save(savepath)

vpype_commands = 'reloop linesimplify --tolerance 0.01mm linemerge --tolerance 0.1mm reloop linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write --page-format {page_format} {savepath}'

os.system(vpype_str)

# try 12

In [ ]:
page_x_inches: float = 11 # inches
page_y_inches: float = 8.5 # inches
border:float = 0.

perlin_grid_params = {
    'xstep':3,
    'ystep':3,
    'lod':10,
    'falloff':None,
    'noise_scale':0.0083,
    'noiseSeed':3
}
buffer_style = 3

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds
brad = np.min([gp.get_width(drawbox), gp.get_height(drawbox)])

start_area = drawbox.buffer(-10, cap_style=3, join_style=3)
pg = gp.PerlinGrid(start_area, **perlin_grid_params)
xcs, ycs = gp.overlay_grid(start_area, xstep=15, ystep=55)
start_pts = [Point(x,y) for x,y in itertools.product(xcs, ycs)]
start_pts = [p for p in start_pts if start_area.contains(p)]

In [ ]:
f,ax = plt.subplots(figsize=(6,6))
ax.quiver(np.cos(pg.a), np.sin(pg.a), scale=50)

In [ ]:
gpolys = []
for p in start_pts:

    pc = ParticleCluster(pos=p, perlin_grid=pg)
    pc.gen_start_pts_gaussian(n_particles=60, xscale=7, yscale=12)
    pc.init_particles(start_bounds=start_area)
    n_steps = np.random.randint(low=1, high=15, size=len(pc.particles))
    pc.step(n_steps)
    
    buffer_distances = np.random.uniform(low=0.3, high=6.3) + np.random.uniform(low=0., high=0.5, size=len(pc.lines))
    polys = []
    for line, bd in zip(pc.lines, buffer_distances):
        bd = np.interp(line.centroid.x, [10, 270], [0.1, 3])
        poly = line.buffer(bd,
            cap_style=buffer_style,
            join_style=buffer_style)
        polys.append(poly)

    P = gp.Poly(so.unary_union(polys))

    stp = ScaleTransPrms(
        d_buffer=np.random.uniform(low=-0.65, high=-0.3), 
        angles=-90,

    )
    stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)
    P.fill_scale_trans(**stp.prms)
    gpolys.append(P)

In [ ]:
zorder = np.random.permutation(len(gpolys))

for z, gpoly in zip(zorder, gpolys):
    gpoly.z = z

for gp0, gp1 in itertools.combinations(gpolys, r=2):
    try:
        overlaps = gp0.p.overlaps(gp1.p)
    except :
        gp0.p = gp0.p.buffer(1e-6)
        gp1.p = gp1.p.buffer(1e-6)
        overlaps = gp0.p.overlaps(gp1.p)
    if overlaps:
        if gp0.z > gp1.z:
            gp1.p = occlude(top=gp0.p, bottom=gp1.p)
        elif gp0.z < gp1.z:
            gp0.p = occlude(top=gp1.p, bottom=gp0.p)

In [ ]:
ifills = []
for p in gpolys:
    try:
        ifills.append(p.intersection_fill)
    except:
        pass

splits = utils.random_split(ifills, n_layers=5)
layers = [gp.merge_LineStrings(split) for split in splits]

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)
sk.display(color_mode='none')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0101_perlin_flow_erode_frays_occlude.svg'

sk.save(savepath)

vpype_commands = 'reloop linesimplify --tolerance 0.01mm linemerge --tolerance 0.1mm reloop linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write --page-format {page_format} {savepath}'

os.system(vpype_str)

# try 13

In [ ]:
page_x_inches: float = 11 # inches
page_y_inches: float = 8.5 # inches
border:float = 0.

perlin_grid_params = {
    'xstep':1,
    'ystep':1,
    'lod':10,
    'falloff':None,
    'noise_scale':0.0193,
    'noiseSeed':3
}
buffer_style = 3

In [ ]:
px = utils.DistanceConverter(page_x_inches, 'inches').mm
py = utils.DistanceConverter(page_y_inches, 'inches').mm
page_format = f'{px}mmx{py}mm'
drawbox = sg.box(border, border, px-border, py-border)

xmin, ymin, xmax, ymax = drawbox.bounds
brad = np.min([gp.get_width(drawbox), gp.get_height(drawbox)])

start_area = drawbox.buffer(-10, cap_style=3, join_style=3)
pg = gp.PerlinGrid(start_area, **perlin_grid_params)
xcs, ycs = gp.overlay_grid(start_area, xstep=20, ystep=75)
start_pts = [Point(x,y) for x,y in itertools.product(xcs, ycs)]
start_pts = [p for p in start_pts if start_area.contains(p)]

In [ ]:
# f,ax = plt.subplots(figsize=(6,6))
# ax.quiver(np.cos(pg.a), np.sin(pg.a), scale=50)

In [ ]:
gpolys = []
for p in start_pts:

    pc = ParticleCluster(pos=p, perlin_grid=pg)
    pc.gen_start_pts_gaussian(n_particles=60, xscale=8, yscale=13)
    pc.init_particles(start_bounds=start_area)
    n_steps = int(np.interp(pc.pos.centroid.x, [1, 270], [1, 30]))
#     n_steps = np.random.randint(low=1, high=15, size=len(pc.particles))
    pc.step(n_steps)
    
    buffer_distances = np.random.uniform(low=0.3, high=6.3) + np.random.uniform(low=0., high=0.5, size=len(pc.lines))
    polys = []
    for line, bd in zip(pc.lines, buffer_distances):
        bd = np.interp(line.centroid.x, [10, 270], [0.1, 3.5])
        poly = line.buffer(bd,
            cap_style=buffer_style,
            join_style=buffer_style)
        polys.append(poly)

    P = gp.Poly(so.unary_union(polys))

    stp = ScaleTransPrms(
        d_buffer=np.random.uniform(low=-0.4, high=-0.2), 
        angles=-60,

    )
    stp.d_buffers += np.random.uniform(-0.06, 0.06, size=stp.d_buffers.shape)
    P.fill_scale_trans(**stp.prms)
    gpolys.append(P)

In [ ]:
zorder = np.random.permutation(len(gpolys))

for z, gpoly in zip(zorder, gpolys):
    gpoly.z = z

for gp0, gp1 in itertools.combinations(gpolys, r=2):
    try:
        overlaps = gp0.p.overlaps(gp1.p)
    except :
        gp0.p = gp0.p.buffer(1e-6)
        gp1.p = gp1.p.buffer(1e-6)
        overlaps = gp0.p.overlaps(gp1.p)
    if overlaps:
        if gp0.z > gp1.z:
            gp1.p = occlude(top=gp0.p, bottom=gp1.p)
        elif gp0.z < gp1.z:
            gp0.p = occlude(top=gp1.p, bottom=gp0.p)

In [ ]:
ifills = []
for p in gpolys:
    try:
        ifills.append(p.intersection_fill)
    except:
        pass

splits = utils.random_split(ifills, n_layers=5)
layers = [gp.merge_LineStrings(split) for split in splits]

In [ ]:
sk = vsketch.Vsketch()
sk.size(page_format)
sk.scale('1mm')
sk.penWidth('0.25mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)
sk.display(color_mode='none')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0102_perlin_flow_erode_frays_occlude.svg'

sk.save(savepath)

In [ ]:
vpype_commands = 'reloop linesimplify --tolerance 0.01mm linemerge --tolerance 0.1mm reloop linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write {savepath}'

os.system(vpype_str)